In [2]:
from statsbombpy import sb
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score,cross_val_predict, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor, XGBClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, classification_report
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import warnings

In [9]:

# Load your dataset
data = pd.read_csv('Statsbomb Data.csv')

# Function to handle unhashable types for one-hot encoding
def make_hashable(x):
    if isinstance(x, list):
        return str(x)  # Convert lists to string to make them hashable
    return x

# Apply this function to all columns that might contain unhashable types
for col in data.columns:
    data[col] = data[col].apply(make_hashable)

# Define functions to calculate additional metrics if relevant
def calculate_distance_to_goal(shot_location):
    if isinstance(shot_location, str):
        shot_location = eval(shot_location)  # Convert string back to list
    goal_x, goal_y = 120, 40  # Coordinates of the center of the goal line
    return np.sqrt((goal_x - shot_location[0])**2 + (goal_y - shot_location[1])**2)

def calculate_angle_to_goal(shot_location):
    if isinstance(shot_location, str):
        shot_location = eval(shot_location)  # Convert string back to list
    goal_y1, goal_y2 = 36, 44  # y-coordinates of the goalposts
    angle1 = np.arctan2(goal_y1 - shot_location[1], 120 - shot_location[0])
    angle2 = np.arctan2(goal_y2 - shot_location[1], 120 - shot_location[0])
    return abs(angle1 - angle2)

# Calculate metrics and add to data
data['distance_to_goal'] = data['location'].apply(calculate_distance_to_goal)
data['angle_to_goal'] = data['location'].apply(calculate_angle_to_goal)

# Convert all categorical variables using one-hot encoding
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# Calculate correlations with 'shot_statsbomb_xg'
correlation_matrix = data.corr()
statsbomb_xg_correlations = correlation_matrix['shot_statsbomb_xg'].sort_values(ascending=False)

# Print the correlations
print(statsbomb_xg_correlations)

# Save to CSV if needed
statsbomb_xg_correlations.to_csv('complete_correlation_with_statsbomb_xg.csv')


shot_statsbomb_xg         1.000000
shot_type_Penalty         0.782955
location_[108.1, 40.1]    0.593886
shot_outcome_Goal         0.535508
angle_to_goal             0.513792
                            ...   
shot_outcome_Off T       -0.123822
duration                 -0.154096
under_pressure           -0.163376
distance_to_goal         -0.450499
shot_type_Open Play      -0.580484
Name: shot_statsbomb_xg, Length: 9759, dtype: float64


In [ ]:
#CODE TO EXTRACT DATA
# # get a list of all competitions
competitions = sb.competitions()
competitions = competitions[competitions['competition_gender'] == 'female']

#using the season id and competition id from competitions, get all match ids from competition
matches = sb.matches(competition_id=72, season_id=107)
match_ids = matches['match_id']
match_ids

# filter shots data to only include columns of interest
cols = ['id', 'player', 'team', 'minute', 'location', 'shot_type', 'shot_outcome', 'under_pressure', 'shot_body_part', 'shot_end_location',
        'shot_first_time', 'shot_freeze_frame', 'shot_key_pass_id', 'shot_one_on_one', 'shot_outcome', 'shot_technique', 'shot_first_time', 'shot_deflected', 'pass_assisted_shot_id', 
        'pass_shot_assist', 'pass_goal_assist', 'pass_type','pass_end_location', 'pass_assisted_shot_id', 'goalkeeper_end_location', 'goalkeeper_position',
        'goalkeeper_outcome', 'goalkeeper_technique', 'period', 'play_pattern', 'shot_statsbomb_xg']

#for each match, get all shots data
shots_data = pd.DataFrame()
for i in range(0, len(match_ids)):
    #try:
    print(match_ids[i])
    match_events = sb.events(match_id=match_ids[i])
    shots = match_events[match_events['type'] == 'Shot']
    #shots = shots[cols]
    shots_data = pd.concat([shots_data, shots])
shots_df = pd.DataFrame(shots_data)
shots_df.to_csv("shots_data.csv")


In [3]:
#Create Model dataset from statsbomb data
# Load the dataset
file_path = 'Statsbomb Data.csv'
data = pd.read_csv(file_path)

# Define the goal post coordinates considering the goal width
goal_center_x = 120
goal_center_y = 40.3
goal_width = 7.32

left_goal_post_x = 120
left_goal_post_y = goal_center_y - (goal_width / 2)
right_goal_post_x = 120
right_goal_post_y = goal_center_y + (goal_width / 2)

# Function to calculate the distance to goal
def calculate_distance_to_goal(location):
    x, y = eval(location)
    return np.sqrt((goal_center_x - x) ** 2 + (goal_center_y - y) ** 2)

# Function to calculate the angle to both goal posts
def calculate_angle_to_goal_posts(location):
    x, y = eval(location)
    left_post_angle = np.arctan2(left_goal_post_y - y, left_goal_post_x - x)
    right_post_angle = np.arctan2(right_goal_post_y - y, right_goal_post_x - x)
    return np.degrees(right_post_angle - left_post_angle)

# Function to normalize the shot body part
def normalize_shot_body_part(shot_body_part):
    if shot_body_part in ['Left Foot', 'Right Foot']:
        return 'Foot'
    return shot_body_part

# Function to classify positions
def classify_position(position):
    forward_positions = [
        'Center Forward', 'Left Wing', 'Right Wing', 'Left Center Forward', 'Right Center Forward'
    ]
    midfielder_positions = [
        'Center Attacking Midfield', 'Left Midfield', 'Right Midfield', 'Left Center Midfield', 
        'Right Center Midfield', 'Center Defensive Midfield', 'Left Defensive Midfield', 
        'Right Defensive Midfield', 'Left Attacking Midfield', 'Right Attacking Midfield'
    ]
    defender_positions = [
        'Left Back', 'Right Back', 'Left Center Back', 'Right Center Back', 'Center Back', 
        'Left Wing Back', 'Right Wing Back'
    ]
    goalkeeper_positions = ['Goalkeeper']

    if position in forward_positions:
        return 'Forward'
    elif position in midfielder_positions:
        return 'Midfielder'
    elif position in defender_positions:
        return 'Defender'
    elif position in goalkeeper_positions:
        return 'Goalkeeper'
    else:
        return 'Unknown'

# Select and process the relevant columns
processed_data = data[['location', 'play_pattern', 'shot_type', 'minute', 'period', 'under_pressure', 'shot_one_on_one', 'shot_first_time', 'shot_technique', 'shot_statsbomb_xg', 'player', 'position', 'possession_team', 'shot_outcome']].copy()

# Calculate distance to goal
processed_data['distance_to_goal'] = data['location'].apply(calculate_distance_to_goal).round(5)

# Calculate angle to goal posts
processed_data['angle_to_goal'] = data['location'].apply(calculate_angle_to_goal_posts).round(5)

# Calculate shot speed
processed_data['shot_speed'] = (processed_data['distance_to_goal'] / data['duration']).round(5)

# Normalize shot body part
processed_data['shot_body_part'] = data['shot_body_part'].apply(normalize_shot_body_part)

# Classify positions
processed_data['position'] = data['position'].apply(classify_position)

# Rename the statsbomb_xg column
processed_data.rename(columns={'shot_statsbomb_xg': 'statsbomb_xg'}, inplace=True)

# Reorder columns as specified, adding team (possession_team) at the start
processed_data = processed_data[['possession_team', 'player', 'position', 'location', 'distance_to_goal', 'angle_to_goal', 'shot_speed', 'play_pattern', 'shot_type', 'minute', 'period', 'under_pressure', 'shot_body_part', 'shot_one_on_one', 'shot_first_time', 'shot_technique', 'statsbomb_xg', 'shot_outcome']]

# Save the processed data to a new CSV file
processed_data.to_csv('model_data.csv', index=False)

# Display the first few rows of the processed dataset
processed_data.head()


,possession_team,player,position,location,distance_to_goal,angle_to_goal,shot_speed,play_pattern,shot_type,minute,period,under_pressure,shot_body_part,shot_one_on_one,shot_first_time,shot_technique,statsbomb_xg,shot_outcome
0,Morocco Women's,Ibtissam Jraïdi,Forward,"[115.6, 23.1]",17.75387,6.09144,37.28771,Regular Play,Open Play,0,1,False,Foot,True,False,Normal,0.011643,Saved
1,Colombia Women's,Mayra Tatiana Ramírez Ramírez,Forward,"[113.6, 54.5]",15.57562,11.55180,22.60042,Regular Play,Open Play,10,1,True,Foot,False,True,Normal,0.017579,Off T
2,Morocco Women's,Ibtissam Jraïdi,Forward,"[107.8, 33.7]",13.87083,26.51428,11.62217,Regular Play,Open Play,12,1,False,Head,False,False,Normal,0.062036,Saved
3,Colombia Women's,María Catalina Usme Pineda,Midfielder,"[103.2, 74.4]",38.01381,4.90953,18.59899,From Free Kick,Free Kick,16,1,False,Foot,False,False,Normal,0.005588,Off T
4,Colombia Women's,Daniela Montoya Quiróz,Midfielder,"[111.0, 39.3]",9.05539,43.83945,429.42998,From Throw In,Open Play,19,1,True,Head,False,False,Normal,0.169690,Saved


In [56]:
# Load the processed dataset
file_path = 'model_data.csv'
data = pd.read_csv(file_path)

# Create the target variable based on shot_outcome
data['shot_outcome'] = data['shot_outcome'].apply(lambda x: 1 if x == 'Goal' else 0)

# Drop the columns that are not needed for the model
data = data.drop(columns=['location', 'period', 'statsbomb_xg', 'possession_team', 'player'])

# Handle categorical variables including 'position'
data = pd.get_dummies(data, drop_first=True)

# Define the features and target variable
X = data.drop(columns=['shot_outcome'])
y = data['shot_outcome']

# Scale the features using Standard scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Function to evaluate models using cross-validation
def evaluate_model_cv(model, X, y):
    y_pred_prob = cross_val_predict(model, X, y, cv=10, method='predict_proba')[:, 1]
    log_loss_score = log_loss(y, y_pred_prob)
    roc_auc = roc_auc_score(y, y_pred_prob)
    y_pred = cross_val_predict(model, X, y, cv=10)
    report = classification_report(y, y_pred)
    
    return log_loss_score, roc_auc, report

# Logistic Regression
lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
log_loss_lr, roc_auc_lr, report_lr = evaluate_model_cv(lr_model, X_scaled, y)
print(f'Logistic Regression - Log Loss (10-fold CV): {log_loss_lr}')
print(f'Logistic Regression - ROC AUC Score (10-fold CV): {roc_auc_lr}')
print('Logistic Regression - Classification Report:')
print(report_lr)

# XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
log_loss_xgb, roc_auc_xgb, report_xgb = evaluate_model_cv(xgb_model, X_scaled, y)
print(f'XGBoost - Log Loss (10-fold CV): {log_loss_xgb}')
print(f'XGBoost - ROC AUC Score (10-fold CV): {roc_auc_xgb}')
print('XGBoost - Classification Report:')
print(report_xgb)

# CatBoost
catboost_model = CatBoostClassifier(verbose=0)
log_loss_cat, roc_auc_cat, report_cat = evaluate_model_cv(catboost_model, X_scaled, y)
print(f'CatBoost - Log Loss (10-fold CV): {log_loss_cat}')
print(f'CatBoost - ROC AUC Score (10-fold CV): {roc_auc_cat}')
print('CatBoost - Classification Report:')
print(report_cat)

# Random Forest
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
log_loss_rf, roc_auc_rf, report_rf = evaluate_model_cv(rf_model, X_scaled, y)
print(f'Random Forest - Log Loss (10-fold CV): {log_loss_rf}')
print(f'Random Forest - ROC AUC Score (10-fold CV): {roc_auc_rf}')
print('Random Forest - Classification Report:')
print(report_rf)

# LightGBM
lgb_model = lgb.LGBMClassifier()
log_loss_lgb, roc_auc_lgb, report_lgb = evaluate_model_cv(lgb_model, X_scaled, y)
print(f'LightGBM - Log Loss (10-fold CV): {log_loss_lgb}')
print(f'LightGBM - ROC AUC Score (10-fold CV): {roc_auc_lgb}')
print('LightGBM - Classification Report:')
print(report_lgb)


Logistic Regression - Log Loss (10-fold CV): 0.3054622738179438
Logistic Regression - ROC AUC Score (10-fold CV): 0.9386043943269007
Logistic Regression - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.88      0.93      1496
           1       0.48      0.88      0.62       184

    accuracy                           0.88      1680
   macro avg       0.73      0.88      0.77      1680
weighted avg       0.93      0.88      0.90      1680

XGBoost - Log Loss (10-fold CV): 0.2421192114885202
XGBoost - ROC AUC Score (10-fold CV): 0.9329698035340619
XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1496
           1       0.70      0.60      0.65       184

    accuracy                           0.93      1680
   macro avg       0.83      0.79      0.80      1680
weighted avg       0.92      0.93      0.93      1680

CatBoost - Log Loss (10-fo

In [ ]:
#TUNED MODELS

warnings.filterwarnings('ignore', category=UserWarning)  # Suppress specific warnings

# Load the processed dataset
file_path = 'model_data.csv'
data = pd.read_csv(file_path)

# Create the target variable based on shot_outcome
data['shot_outcome'] = data['shot_outcome'].apply(lambda x: 1 if x == 'Goal' else 0)

# Drop the columns that are not needed for the model
data = data.drop(columns=['location', 'period', 'statsbomb_xg', 'possession_team', 'player'])

# Handle categorical variables including 'position'
data = pd.get_dummies(data, drop_first=True)

# Define the features and target variable
X = data.drop(columns=['shot_outcome'])
y = data['shot_outcome']

# Scale the features using Standard scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Function to evaluate models using cross-validation
def evaluate_model_cv(model, X, y):
    y_pred_prob = cross_val_predict(model, X, y, cv=10, method='predict_proba')[:, 1]
    log_loss_score = log_loss(y, y_pred_prob)
    roc_auc = roc_auc_score(y, y_pred_prob)
    y_pred = cross_val_predict(model, X, y, cv=10)
    report = classification_report(y, y_pred)
    
    return log_loss_score, roc_auc, report

# Hyperparameter grids
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6]
}

param_grid_catboost = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6]
}

param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

param_grid_lgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [31, 50],
    'min_child_samples': [20, 30],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Logistic Regression
lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
grid_search_lr = GridSearchCV(lr_model, param_grid_lr, cv=10, scoring='neg_log_loss')
log_loss_lr, roc_auc_lr, report_lr = evaluate_model_cv(grid_search_lr, X_scaled, y)
print(f'Logistic Regression - Log Loss (10-fold CV): {log_loss_lr}')
print(f'Logistic Regression - ROC AUC Score (10-fold CV): {roc_auc_lr}')
print('Logistic Regression - Classification Report:')
print(report_lr)

# XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=10, scoring='neg_log_loss')
log_loss_xgb, roc_auc_xgb, report_xgb = evaluate_model_cv(grid_search_xgb, X_scaled, y)
print(f'XGBoost - Log Loss (10-fold CV): {log_loss_xgb}')
print(f'XGBoost - ROC AUC Score (10-fold CV): {roc_auc_xgb}')
print('XGBoost - Classification Report:')
print(report_xgb)

# CatBoost
catboost_model = CatBoostClassifier(verbose=0)
grid_search_catboost = GridSearchCV(catboost_model, param_grid_catboost, cv=10, scoring='neg_log_loss')
log_loss_cat, roc_auc_cat, report_cat = evaluate_model_cv(grid_search_catboost, X_scaled, y)
print(f'CatBoost - Log Loss (10-fold CV): {log_loss_cat}')
print(f'CatBoost - ROC AUC Score (10-fold CV): {roc_auc_cat}')
print('CatBoost - Classification Report:')
print(report_cat)

# Random Forest
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=10, scoring='neg_log_loss')
log_loss_rf, roc_auc_rf, report_rf = evaluate_model_cv(grid_search_rf, X_scaled, y)
print(f'Random Forest - Log Loss (10-fold CV): {log_loss_rf}')
print(f'Random Forest - ROC AUC Score (10-fold CV): {roc_auc_rf}')
print('Random Forest - Classification Report:')
print(report_rf)

# LightGBM
lgb_model = lgb.LGBMClassifier()
grid_search_lgb = GridSearchCV(lgb_model, param_grid_lgb, cv=10, scoring='neg_log_loss')
log_loss_lgb, roc_auc_lgb, report_lgb = evaluate_model_cv(grid_search_lgb, X_scaled, y)
print(f'LightGBM - Log Loss (10-fold CV): {log_loss_lgb}')
print(f'LightGBM - ROC AUC Score (10-fold CV): {roc_auc_lgb}')
print('LightGBM - Classification Report:')
print(report_lgb)
